# Istunto 1 – Chat Bootstrap (Foundry Local)

Tämä muistikirja käynnistää Foundry Localin, lataa valitun mallialiasin ja suorittaa sekä tavallisen että suoratoistavan chat-täydennyksen.


# Tilanne
Tässä osiossa käydään läpi ehdottoman vähimmäismäärä, joka tarvitaan pienen paikallisen kielimallin vastaamiseen Foundry Localin kautta. Teet seuraavat asiat:
- Asennat SDK:n / asiakasohjelman riippuvuudet.
- Alustat Foundry Local -hallinnan valitulle aliakselle (oletus: `phi-3.5-mini`).
- Sovellat puolustavaa monkey-patchia, joka sallii valinnaiset kentät mallin metatiedoissa.
- Lähetät tavallisen chat-vastauspyynnön.
- Striimaat vastauksen token kerrallaan.

Tavoitteena on vahvistaa paikallinen suoritusympäristösi ja verkkopolku ennen siirtymistä RAG:iin, reititykseen tai agenteihin.


### Selitys: Riippuvuuksien asennus
Asentaa Python-paketit, jotka tarvitaan tähän minimaaliseen keskusteluprosessiin:
- `foundry-local-sdk`: Hallitse paikallisia malleja ja palvelun elinkaarta.
- `openai`: Käytännöllinen asiakasrajapinta keskusteluvastauksille.
- `rich`: Kauniimpi tulostus selkeämmän notebook-näkymän saavuttamiseksi.

Uudelleenkäynnistys on turvallista (idempotentti). Ohita, jos ympäristössäsi on jo nämä.


In [1]:
# Install required libraries (idempotent)
!pip install -q foundry-local-sdk openai rich

### Selitys: Keskeiset tuonnit
Tuodaan moduuleja, joita käytetään koko muistikirjassa:
- `FoundryLocalManager` paikallisen mallin käyttöliittymän hallintaan.
- `OpenAI`-asiakas, jotta voimme hyödyntää tuttua chat-kompletion API-pintaa.
- `rich.print` tyyliteltyä tulostusta varten.

Tässä ei tehdä verkkokutsuja—vain nimialue valmistellaan.


In [2]:
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI
from rich import print

### Selitys: Managerin alustaminen ja metadatan korjaus
Alustaa `FoundryLocalManager` valitulle aliasille ja soveltaa suojaavaa monkey-patchia käsittelemään palveluvastauksia, joissa `promptTemplate` voi olla `null`.

Keskeiset tulokset:
- Vahvistaa palvelun tilan ja päätepisteen.
- Listaa välimuistissa olevat mallit (tarkistaa paikallisen tallennuksen).
- Selvittää aliasille konkreettisen mallin tunnuksen (käytetään myöhemmissä chat-kutsuissa).

Jos kohtaat validointiongelmia palvelun metadatassa, tämä malli näyttää, kuinka voit puhdistaa tiedot ilman SDK:n haaroittamista.


In [3]:
# Catalog-safe manager initialization (handles null promptTemplate values)
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-3.5-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:57127/v1

Cached models:
[
    FoundryModelInfo(
        alias='gpt-oss-20b',
        id='gpt-oss-20b-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/gpt-oss-20b-cuda-gpu/versions/1',
        file_size_mb=9882,
        prompt_template={},
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1',
        file_size_mb=2181,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-cuda-gpu:4',
        version='4',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-cuda-gpu/versions/4',
        file_size_mb=3686,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-cuda-gpu/versions/3',
        file_size_mb=528,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-7b',
        id='qwen2.5-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-coder-7b',
        id='qwen2.5-coder-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-coder-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_star

Using model id: Phi-4-mini-instruct-cuda-gpu:4

### Selitys: Perus Chat Completion
Luo `OpenAI`-yhteensopivan asiakkaan, joka osoittaa paikalliseen Foundry-päätepisteeseen, ja suorittaa yhden ei-streamaavan chat-keskustelun. Tärkeimmät kohdat:
- Varmista, että malli vastaa ilman virheitä.
- Tarkista viive / tulostusmuoto.
- Pidä `max_tokens` maltillisena resurssien säästämiseksi.

Jos tämä epäonnistuu, tarkista uudelleen, että Foundry Local -palvelu on käynnissä ja alias ratkaisee oikein.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':prompt}], max_tokens=120, temperature=0.5)
print(resp.choices[0].message.content)

1. Enhanced Data Privacy: Local inference allows data to be processed locally on a user's device rather than being 
sent to a central server. This reduces the risk of sensitive information being intercepted or exposed during 
transmission, thereby enhancing privacy.

2. Reduced Data Collection: By performing inference locally, the need for extensive data collection is minimized. 
This means that less personal information needs to be stored or transmitted, further protecting user privacy.

### Selitys: Keskustelun suoratoisto
Esimerkki tokenien suoratoistosta, joka parantaa koettua viivettä ja vuorovaikutteista käyttökokemusta. Silmukka tulostaa saapuvat inkrementaaliset muutokset:
- Hyödyllinen keskustelukäyttöliittymissä, joissa varhainen osittainen tulos on tärkeä.
- Mahdollistaa tokenien läpimenon mittaamisen verrattuna täydellisen valmistumisen viiveeseen.

Voit mukauttaa tämän mallin keräämään tokeneita, päivittämään edistymispalkkia tai keskeyttämään sukupolven kesken.


In [5]:
# Streaming example
stream = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}], stream=True, max_tokens=60, temperature=0.4)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

with

a

central

server

.


---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
